In [1]:
import rasterio
import geopandas as gpd
import numpy as np
import os
from shapely.geometry import shape
from rasterio.features import geometry_mask
import fiona
from rasterio.transform import from_origin




In [2]:
tif_path = 'jamaica-03-2022.tif'
shapefile_path = 'jamaica-labels.shp'

In [3]:
# Load the shapefile containing labels
os.environ['SHAPE_RESTORE_SHX'] = 'YES'
gdf = gpd.read_file(shapefile_path)
# labels = gdf['geometry'] 

In [4]:
# shapefile_path = 'path_to_your_shapefile.shp'

# Path to save the output raster
output_raster_path = 'output_raster.tif'

# Load the shapefile's geometries as Shapely objects
shapefile_geoms = []
with fiona.open(shapefile_path, "r") as shapefile:
    for feature in shapefile:
        shapefile_geoms.append(shape(feature['geometry']))

# Calculate the extent and resolution for the raster
xmin, ymin, xmax, ymax = shapefile.bounds
pixel_size = 10  # Choose an appropriate pixel size
width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)

# Create a transform for the raster
transform = from_origin(xmin, ymax, pixel_size, pixel_size)

# Create an empty binary array
binary_array = np.zeros((height, width), dtype=np.uint8)

# Rasterize the shapefile's polygons into the binary array
for geom in shapefile_geoms:
    mask = geometry_mask([geom], out_shape=(height, width), transform=transform, invert=False)
    binary_array[mask] = 1

# Write the binary array as a raster
with rasterio.open(output_raster_path, 'w', driver='GTiff', height=height, width=width, count=1, dtype=np.uint8, crs='EPSG:4326', transform=transform) as dst:
    dst.write(binary_array, 1)

AttributeError: 'NoneType' object has no attribute 'get'

In [ ]:
count_ones = np.count_nonzero(binary_array == 1)

print("Number of ones:", count_ones)

Number of ones: 0


In [ ]:
# Load the GeoTIFF image
with rasterio.open(tif_path) as tif_src:
    tif_transform = tif_src.transform



In [ ]:
# Set the CRS for the GeoDataFrame
gdf = gdf.set_crs(tif_src.crs)
gdf = gdf[gdf.geometry.is_valid]

# Reproject the GeoDataFrame to match the GeoTIFF's coordinate system
gdf_reprojected = gdf.to_crs(tif_src.crs)

# Create an empty binary array
binary_array = np.zeros((tif_src.height, tif_src.width), dtype=np.uint8)

# Load polygons from the GeoDataFrame and assign 1 to pixels within polygons
# cum = 0
for geom in gdf_reprojected.geometry:
    # cum += 1
    mask = geometry_mask([geom], out_shape=(tif_src.height, tif_src.width), transform=tif_transform, invert=True)
    binary_array[mask] = 1

# print(cum)

In [ ]:
with rasterio.open(tif_path) as tif_src:
    # Get the raster transformation
    raster_transform = tif_src.transform

    # Print the transformation matrix
    print("Raster Transformation Matrix:")
    print(raster_transform)

Raster Transformation Matrix:
| 10.00, 0.00, 881520.00|
| 0.00,-10.00, 1983400.00|
| 0.00, 0.00, 1.00|


In [ ]:
count_ones = np.count_nonzero(binary_array == 0)

print("Number of ones:", count_ones)

Number of ones: 0
